# Project 8 : Déployer un modèle dans le cloud
## Partie 2 : Extraction des features dans le cloud 

Ce notebook est déjà lancé par la commande pyspark qui utilise le notebook comme driver, donc le SparkContext est déjà initialisé comme le montre la commande ci dessous 
Dans la première partie on a sauvé un échantillon de nos données sur le S3 d'Amazon, dans cette partie, on va juste pouvoir extraire les features en utilisant SparkDL

In [1]:
sc

<SparkContext master=yarn appName=PySparkShell>

In [2]:
from pyspark.sql.functions import lit
from pyspark.ml.image import ImageSchema
import boto3
from sparkdl import DeepImageFeaturizer 

/home/hadoop/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/hadoop/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/hadoop/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/hadoop/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:529: 

On commence par récupérer la bucket S3, notons que les credentials et la config se trouve dans le répertoire ~/.aws/credentials et ~/.aws/config comme l'expliquait la documentation Amazon S3

In [3]:
s3_client = boto3.client('s3')
s3_resource = boto3.resource('s3')
bucket_name = 'oc-p8-salah'
region = 'us-east-2'

On construit une dataframe spark, avec comme colonne l'image, et bien entendu son label sur l'échantillon d'images qu'on a récupérées

In [4]:
s3_bucket = s3_resource.Bucket(bucket_name)
path_prefix = 's3a://' + bucket_name + '/'
main_df = None
i = 0
for bucket_object in s3_bucket.objects.all():
    if not bucket_object.key.endswith('.jpg'):
        continue
    label = bucket_object.key.split('/')[0]
    img_path = path_prefix + bucket_object.key
    img_df = ImageSchema.readImages(img_path).withColumn('label', lit(label))
    if main_df is None:
        main_df = img_df
    else:
        main_df = main_df.unionAll(img_df)
    print('Done for img {} count = {}'.format(img_path,i))
    i+=1

main_df.show(3)


Done for img s3a://oc-p8-salah/Cherry_Wax_Yellow/126_100.jpg count = 0
Done for img s3a://oc-p8-salah/Cherry_Wax_Yellow/158_100.jpg count = 1
Done for img s3a://oc-p8-salah/Cherry_Wax_Yellow/170_100.jpg count = 2
Done for img s3a://oc-p8-salah/Cherry_Wax_Yellow/31_100.jpg count = 3
Done for img s3a://oc-p8-salah/Cherry_Wax_Yellow/58_100.jpg count = 4
Done for img s3a://oc-p8-salah/Cherry_Wax_Yellow/r_132_100.jpg count = 5
Done for img s3a://oc-p8-salah/Cherry_Wax_Yellow/r_146_100.jpg count = 6
Done for img s3a://oc-p8-salah/Cherry_Wax_Yellow/r_172_100.jpg count = 7
Done for img s3a://oc-p8-salah/Cherry_Wax_Yellow/r_231_100.jpg count = 8
Done for img s3a://oc-p8-salah/Cherry_Wax_Yellow/r_247_100.jpg count = 9
Done for img s3a://oc-p8-salah/Kohlrabi/263_100.jpg count = 10
Done for img s3a://oc-p8-salah/Kohlrabi/318_100.jpg count = 11
Done for img s3a://oc-p8-salah/Kohlrabi/88_100.jpg count = 12
Done for img s3a://oc-p8-salah/Kohlrabi/r_151_100.jpg count = 13
Done for img s3a://oc-p8-sala

Notre dataframe étant prête, il ne reste plus qu'à en extraire les features, pour celà on va utiliser du transfer learning et utiliser le ResNet50 pour extraires les features des images

In [5]:
featurizer = DeepImageFeaturizer(
    inputCol='image',
    outputCol='feature',
    modelName='ResNet50'
)

Regardons quelques échantillons de la dataframe, et aussi son schema

In [6]:
features_df = featurizer.transform(main_df)
features_df.show(3)

+--------------------+-----------------+--------------------+
|               image|            label|             feature|
+--------------------+-----------------+--------------------+
|[s3a://oc-p8-sala...|Cherry_Wax_Yellow|[0.15766188502311...|
|[s3a://oc-p8-sala...|Cherry_Wax_Yellow|[0.35756713151931...|
|[s3a://oc-p8-sala...|Cherry_Wax_Yellow|[0.52175581455230...|
+--------------------+-----------------+--------------------+
only showing top 3 rows



In [7]:
features_df.printSchema()

root
 |-- image: struct (nullable = true)
 |    |-- origin: string (nullable = true)
 |    |-- height: integer (nullable = false)
 |    |-- width: integer (nullable = false)
 |    |-- nChannels: integer (nullable = false)
 |    |-- mode: integer (nullable = false)
 |    |-- data: binary (nullable = false)
 |-- label: string (nullable = false)
 |-- feature: vector (nullable = true)



Finalement reste plus qu'à sauver cette dataframe, on préfère utiliser le format parquet bien utile pour sauver à la fois les images autant qu'objets, ainsi que les features autant que vector

In [ ]:
features_df.write.format('parquet').mode('overwrite').save('s3a://' + bucket_name + '/processed.out')